# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
from scipy.stats import ttest_1samp
import numpy as np
from scipy.stats import norm

from scipy.stats import t
import matplotlib.pyplot as plt
sns.set_style("darkgrid") 

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [80]:
# Your code here:

df = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

In [81]:
df.dtypes

Name                  object
Job Titles            object
Department            object
Full or Part-Time     object
Salary or Hourly      object
Typical Hours        float64
Annual Salary        float64
Hourly Rate          float64
dtype: object

Examine the `salaries` dataset using the `head` function below.

In [82]:
# Your code here:

df.head(100)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86
...,...,...,...,...,...,...,...,...
95,"ACKLIN, QIANA D",CROSSING GUARD,OEMC,P,Hourly,20.0,NaN,17.68
96,"ACOSTA, CESAR I",STEAMFITTER,WATER MGMNT,F,Hourly,40.0,NaN,47.50
97,"ACOSTA, JAIME A",POLICE OFFICER,POLICE,F,Salary,NaN,72510.0,NaN
98,"ACOSTA, JESSE A",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [5]:
# Your code here:

null_cols = df.isnull().sum()
null_cols[null_cols > 0]

Typical Hours    25161
Annual Salary     8022
Hourly Rate      25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [6]:
# Your code here:

df_salary = df.loc[(df['Salary or Hourly'] == 'Salary')].count()

In [7]:
df_salary

Name                 25161
Job Titles           25161
Department           25161
Full or Part-Time    25161
Salary or Hourly     25161
Typical Hours            0
Annual Salary        25161
Hourly Rate              0
dtype: int64

In [8]:
df_hourly = df.loc[(df['Salary or Hourly'] == 'Hourly')].count()

In [9]:
df_hourly

Name                 8022
Job Titles           8022
Department           8022
Full or Part-Time    8022
Salary or Hourly     8022
Typical Hours        8022
Annual Salary           0
Hourly Rate          8022
dtype: int64

In [10]:
df['Salary or Hourly'].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [11]:
# Your code here:

df.groupby(['Department']).count()

,Name,Job Titles,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
Department,,,,,,,
ADMIN HEARNG,39,39,39,39,0,39,0
ANIMAL CONTRL,81,81,81,81,19,62,19
AVIATION,1629,1629,1629,1629,1082,547,1082
BOARD OF ELECTION,107,107,107,107,0,107,0
BOARD OF ETHICS,8,8,8,8,0,8,0
BUDGET & MGMT,46,46,46,46,2,44,2
BUILDINGS,269,269,269,269,0,269,0
BUSINESS AFFAIRS,171,171,171,171,7,164,7
CITY CLERK,84,84,84,84,0,84,0


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [12]:
from scipy.stats import t

In [13]:
# Your code here:
df_hourly = df.loc[(df['Salary or Hourly'] == 'Hourly')]
df_hourly

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86
6,"ABBATACOLA, ROBERT J",ELECTRICAL MECHANIC,AVIATION,F,Hourly,40.0,NaN,46.10
7,"ABBATE, JOSEPH L",POOL MOTOR TRUCK DRIVER,STREETS & SAN,F,Hourly,40.0,NaN,35.60
10,"ABBOTT, BETTY L",FOSTER GRANDPARENT,FAMILY & SUPPORT,P,Hourly,20.0,NaN,2.65
18,"ABDULLAH, LAKENYA N",CROSSING GUARD,OEMC,P,Hourly,20.0,NaN,17.68
...,...,...,...,...,...,...,...,...
33164,"ZUREK, FRANCIS",ELECTRICAL MECHANIC,OEMC,F,Hourly,40.0,NaN,46.10
33168,"ZWARYCZ MANN, IRENE A",CROSSING GUARD,OEMC,P,Hourly,20.0,NaN,17.68
33169,"ZWARYCZ, THOMAS J",POOL MOTOR TRUCK DRIVER,WATER MGMNT,F,Hourly,40.0,NaN,35.60
33174,"ZYGADLO, JOHN P",MACHINIST (AUTOMOTIVE),GENERAL SERVICES,F,Hourly,40.0,NaN,46.35


In [14]:
alpha = 0.05 #Not sure in T-Student for = or != shall we use this split??!

In [15]:
sample_mean = df_hourly['Hourly Rate'].mean()
sample_mean

32.78855771628128

In [16]:
mu = 30
sample_stdev = df_hourly['Hourly Rate'].std()
standard_error = sample_stdev / np.sqrt(len(df_hourly))

In [17]:
standard_error

0.1352368565101597

In [18]:
t_value = (sample_mean - mu) / standard_error
print(t_value)

20.619805785501907


In [19]:
degreef = len(df_hourly) - 1
mean, var, skew, kurt = t.stats(degreef, moments='mvsk')

print(var)

1.0002494076568151


In [20]:
t, p = stats.ttest_1samp(df_hourly['Hourly Rate'], mu)

In [21]:
print(t,p)

20.6198057854942 4.3230240486229894e-92


In [22]:
p_round = round(p,10)
p_round

0.0

In [23]:
#P-value is less than 0,05, thereby we can reject the null hipoteshis, the mean of the population is not 30. 

In [24]:
alpha = 0.025
alpha = 1 - alpha
alpha = t.ppf(alpha, degreef)
print(alpha)

AttributeError: 'numpy.float64' object has no attribute 'ppf'

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [25]:
df_salary_police = df.loc[(df['Department'] == 'POLICE') & df['Annual Salary'].notnull()]

In [26]:
df_salary_police['Annual Salary'].mean()

86486.41450313339

In [27]:
null_cols = df_salary_police.isnull().sum()
null_cols[null_cols > 0]

Typical Hours    13404
Hourly Rate      13404
dtype: int64

In [32]:
alpha=0.025
mu = 86000


In [33]:
# Your code here:
t, p = stats.ttest_1samp(df_salary_police['Annual Salary'], mu)
print(t,p)

3.081997005712994 0.0020603403550965137


In [34]:
p/2 <= alpha
 

True

Using the `crosstab` function, find the department that has the most hourly workers. 

In [35]:
# Your code here:

pd.crosstab(df['Department'], df['Salary or Hourly']).sort_values(by='Hourly', ascending=False)

Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336
WATER MGMNT,1513,366
OEMC,1273,829
AVIATION,1082,547
GENERAL SERVICES,765,215
TRANSPORTN,725,415
PUBLIC LIBRARY,299,716
FAMILY & SUPPORT,287,328
CITY COUNCIL,64,347


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [36]:
# Your code here:
df_salary_street = df.loc[(df['Department'] == 'STREETS & SAN') & df['Hourly Rate'].notnull()]


In [37]:
mu = 35

In [38]:
df_salary_street['Hourly Rate'].mean()

33.728378088076845

In [39]:
t, p = stats.ttest_1samp(df_salary_street['Hourly Rate'], mu)

In [40]:
t, p

(-9.567447887848152, 3.3378530564707717e-21)

# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [48]:
# Your code here:
sample_mean = df_hourly['Hourly Rate'].mean()
sample_mean

32.78855771628128

In [49]:
df_t = len(df_hourly)-1

In [50]:
confidence_level = 0.95

In [61]:
t = stats.t.interval(confidence_level, df=df_t)[1]

In [62]:
t

1.9602597859561675

In [171]:
std_error = stats.sem(df_hourly['Hourly Rate'], axis=None, ddof=-1)

In [172]:
std_error

0.13521999931389467

In [173]:
con_int = [sample_mean - (t * std_error) , sample_mean + (t * std_error)]
print('With ' + str(confidence_level * 100) + '% confidence, the true mean lies within the inverval:', con_int)

With 95.0% confidence, the true mean lies within the inverval: [32.523491389369234, 33.05362404319333]


In [178]:
import scipy

In [179]:
hourly = df['Hourly Rate'].dropna()
degrees_freedom = hourly.size - 1
sample_mean = np.mean(hourly)
sample_standard_error = scipy.stats.sem(hourly)

confidence_interval = scipy.stats.t.interval(0.95, degrees_freedom, sample_mean, sample_standard_error)
confidence_interval

(32.52345834488529, 33.05365708767727)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [193]:
import math

In [201]:
import statistics

In [67]:
# Your code here:
sample_mean1 = df_salary_police['Annual Salary'].mean()

In [68]:
sample_mean1

86486.41450313339

In [72]:
df2 = len(df_salary_police['Annual Salary'])-1

In [73]:
df2

13403

In [205]:
std1 = df_salary_police['Annual Salary'].std()
std1

18272.228293992157

In [197]:
math.sqrt(std1)

135.17480643223485

In [202]:
var1 = statistics.variance(df_salary_police['Annual Salary'])

In [203]:
var1

333874326.8278013

In [204]:
math.sqrt(var1)

18272.22829399308

In [182]:
std_error = stats.sem(df_salary_police['Annual Salary'], axis=None, ddof=-1)

In [183]:
std_error

157.8126827599245

In [188]:
std_error2 = stats.sem(df_salary_police['Annual Salary'])

In [189]:
std_error2

157.82445675052244

In [184]:
z = stats.norm.interval(confidence_level)[1]

In [185]:
z

1.959963984540054

In [186]:
con_int = [sample_mean1 - (z * std_error) , sample_mean1 + (z * std_error)]
print('With ' + str(confidence_level * 100) + '% confidence, the true mean lies within the inverval:', con_int)

With 95.0% confidence, the true mean lies within the inverval: [86177.10732862029, 86795.72167764649]


In [187]:
confidence_interval = scipy.stats.t.interval(0.95, df2, sample_mean1, std_error)
confidence_interval

(86177.07939399949, 86795.74961226729)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [98]:
import numpy as np
from statsmodels.stats.proportion import proportions_ztest

In [99]:
# Your code here:

df.head(100)

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86
...,...,...,...,...,...,...,...,...
95,"ACKLIN, QIANA D",CROSSING GUARD,OEMC,P,Hourly,20.0,NaN,17.68
96,"ACOSTA, CESAR I",STEAMFITTER,WATER MGMNT,F,Hourly,40.0,NaN,47.50
97,"ACOSTA, JAIME A",POLICE OFFICER,POLICE,F,Salary,NaN,72510.0,NaN
98,"ACOSTA, JESSE A",POLICE OFFICER,POLICE,F,Salary,NaN,93354.0,NaN


In [100]:
df['Salary or Hourly'].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

In [101]:
hourly_p = 8022

In [102]:
hourly_p

8022

In [103]:
count = len(df['Salary or Hourly'])

In [104]:
count

33183

In [166]:
value = 0.25

In [167]:
propor = 8022/33183

In [168]:
propor

0.24175029382515142

In [169]:
stat, pval = proportions_ztest(hourly_p, count, value)

In [170]:
stat,pval

(-3.5099964213703005, 0.0004481127249057967)

In [ ]:
#We reject H0, the % of hourly workers differs from 25%.